# Questao 1
O conjunto de microdados do ENEM 2022 possui 3.476.105 linhas, uma numerosidade muito grande para nosso contexto. Sua primeira tarefa é fazer a leitura do arquivo CSV por blocos e então realizar filtragem e amostragem em cada bloco para diminuir a numerosidade, salvando ao final um CSV com numerosidade reduzida. Tente alcançar uma tabela entre 50mil e 100mil linhas.
Para leitura por blocos, use a opção chunksize de read_csv

In [2]:
import pandas as pd


db = pd.read_csv('micro_dados/MICRODADOS_ENEM_2022.csv', encoding="cp1252",delimiter=";")
db_ba = db[ db["SG_UF_ESC"]=="BA" ]
db_mg = db[ db["SG_UF_ESC"]=="MG" ]


In [3]:
print(len(db_ba))
print(db_ba.shape)

44483
(44483, 76)


In [4]:
print(len(db_mg))
print(db_mg.shape)

68869
(68869, 76)


In [5]:
db_ba.to_csv("base_dados_ma_ba.csv",encoding="cp1252")

In [ ]:
db_mg.to_csv("base_dados_ma_mg.csv",encoding="cp1252")

# quest 2

A)

resp: idade, tipo escola, Q006,Q024,Q025,sexo, estado civil,Q021, TP_ST_CONCLUSAO,Q005

pq escolheu cada um, pode colocar em bloco de atributo,
